In [ ]:
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, Convolution2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import optimizers
from keras.preprocessing import sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import ModelCheckpoint

In [ ]:
DROPOUT = 0.5
N_CATEGORY = 3
model_input = Input(shape = (3, 227, 227))

# First convolutional Layer (96x11x11)
z = Conv2D(filters = 96, kernel_size = (11,11), strides = (4,4), activation = "relu")(model_input)
z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = BatchNormalization()(z)

# Second convolutional Layer (256x5x5)
z = ZeroPadding2D(padding = (2,2))(z)
z = Convolution2D(filters = 256, kernel_size = (5,5), strides = (1,1), activation = "relu")(z)
z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = BatchNormalization()(z)

# Rest 3 convolutional layers
z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 256, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = Flatten()(z)

z = Dense(4096, activation="relu")(z)
z = Dropout(DROPOUT)(z)

z = Dense(4096, activation="relu")(z)
z = Dropout(DROPOUT)(z)

final_dim = 1 if N_CATEGORY == 2 else N_CATEGORY
final_act = "sigmoid" if N_CATEGORY == 2 else "softmax"
model_output = Dense(final_dim, activation=final_act)(z)

model = Model(model_input, model_output)
model.summary()

In [ ]:
WEIGHT_DECAY = 0.0005
MOMENTUM = 0.9
BATCH_SIZE = 128
LEARNING_RATE = 0.01
DROPOUT = 0.5
ALPHA = 1e-4
BETA = 0.75
n = 5
k = 2

loss_metric = "binary_crossentropy" if N_CATEGORY == 2 else "categorical_crossentropy"
model.compile(loss=loss_metric, metrics=["accuracy"], 
              optimizer=optimizers.SGD(lr=LEARNING_RATE, momentum=MOMENTUM, decay = WEIGHT_DECAY))

In [ ]:
# Data augmentation
train_datagen = ImageDataGenerator(
          rotation_range=40, width_shift_range=0.2,
          height_shift_range=0.2, rescale=1./200,
          shear_range=0.2, zoom_range=0.2,
          horizontal_flip=True, fill_mode='nearest'
           )

validate_datagen = ImageDataGenerator(rescale=1./200)

train_data_dir = 'Data/Train/'
validation_data_dir = 'Data/Validate/'
nb_train_samples = 1159 + 1199 + 1272
nb_validation_samples = 564 + 728 + 773
epochs = 100
batch_size = 40

train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(227, 227),
                    batch_size=batch_size,
                    class_mode='categorical'
                    )

validation_generator = validate_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(227, 227),
                            batch_size=batch_size,
                            class_mode='categorical'
                            )

# Checkpoints
filepath = 'Alexnet-Raw-Checkpoints/weights-improvement-{epoch:02d}-{val_acc:.3f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model.fit_generator(train_generator, 
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,
                    callbacks=callbacks_list
                   )

In [ ]:
model.evaluate_generator(train_generator, 3508)

In [ ]:
model.evaluate_generator(validation_generator, 1978)

In [ ]:
model.metrics_names